In [38]:
# 더미 데이터를 훈련, 테스트 데이터로 나누고 파일로 저장

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. 데이터 불러오기
review_dummy_df = pd.read_csv('../data/review_dummy.txt', sep='\t')

# 데이터 확인
print(" === 데이터 상위 5개 === \n", review_dummy_df.head())
print(" === review_dummy_df.info() ===")
review_dummy_df.info()
print("\n === label 컬럼 확인 === \n", review_dummy_df['label'].value_counts())
print("\n === label 유니크 값 === \n", review_dummy_df['label'].unique())
print('review_dummy_df 데이터 확인')
review_dummy_df

# 2. 잘못 들어간 행 제거
review_dummy_df = review_dummy_df[review_dummy_df['label'] != 'label']
# 4. label 컬럼을 숫자형으로 변환
review_dummy_df['label'] = review_dummy_df['label'].astype(int)

# 5. train / test 분리
train_df, test_df = train_test_split(
    review_dummy_df,
    test_size=0.2,
    random_state=42,
    stratify=review_dummy_df['label']
)
print(f"Train 데이터 개수: {len(train_df)}")
print(f"Test 데이터 개수: {len(test_df)}")

# 8. 저장
train_df.to_csv('../data/train_data.txt', sep='\t', index=False, encoding='utf-8')
test_df.to_csv('../data/test_data.txt', sep='\t', index=False, encoding='utf-8')


 === 데이터 상위 5개 === 
         id                                           document label
0  6270596                                                굳 ㅋ     1
1  9274899                               GDNTOPCLASSINTHECLUB     0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아     0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....     0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??     0
 === review_dummy_df.info() ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200001 entries, 0 to 200000
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200001 non-null  object
 1   document  199993 non-null  object
 2   label     200001 non-null  object
dtypes: object(3)
memory usage: 4.6+ MB

 === label 컬럼 확인 === 
 label
1        100000
0        100000
label         1
Name: count, dtype: int64

 === label 유니크 값 === 
 ['1' '0' 'label']
review_dummy_df 데이터 확인
Train 데이터 개수: 160000
Test

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re

# 학습 데이터 파일 로드
try:
    train_df = pd.read_csv('../data/train_data.txt', sep='\t')
    test_df = pd.read_csv('../data/train_data.txt', sep='\t')
except FileNotFoundError:
    print("Error: 'ratings_train.txt' or 'ratings_test.txt' not found.")
    print("Please download them from https://github.com/e9t/nsmc and place them in the same directory as this script.")
    exit()

# 결측치 제거 (학습을 위해)
train_df = train_df.dropna(how='any')
test_df = test_df.dropna(how='any')

# 텍스트 전처리 (정규표현식)
def clean_text(text):
    text = re.sub(r'[^가-힣\s]', '', text) # 한글과 공백만 남김
    return text
train_df['document'] = train_df['document'].apply(clean_text)
test_df['document'] = test_df['document'].apply(clean_text)

# 모델 학습/테스트를 위한 데이터셋 분리
X_train = train_df['document']
y_train = train_df['label']
X_test = test_df['document']
y_test = test_df['label']

# 텍스트 벡터화 (TF-IDF)
# ngram_range=(1, 2): ‘스토리’, ‘스토리 좋다’ 같은 단어 묶음도 학습.
tfidf_vectorizer = TfidfVectorizer(min_df=3, ngram_range=(1, 2))

# 어휘 학습
# 텍스트 → 숫자 벡터로 바꾸기 위한 단어 사전 생성
tfidf_vectorizer.fit(X_train)
# transform 적용 (어휘 학습 및 변환)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 모델 학습 (로지스틱 회귀)
# 벡터화된 숫자 데이터를 이용해 긍정/부정 분류 모델 학습
model = LogisticRegression(random_state=42, solver='liblinear', C=1.0)
model.fit(X_train_tfidf, y_train)

# 4. 모델 평가
# 테스트 데이터셋으로 모델 성능을 평가합니다.
model_predict = model.predict(X_test_tfidf)

# 정확도 계산 (실제 정답, 모델 예측 결과)
accuracy = accuracy_score(y_test, model_predict)
print(f"모델 정확도 : {accuracy:.4f}")

모델 정확도 : 0.8545


In [4]:
# 모델 평가 지표
print("\n모델 평가 지표:")
print(classification_report(y_test, model_predict,
                            target_names=['긍정(Negative)(0)', '부정(Positive)(1)']))



모델 평가 지표:
                 precision    recall  f1-score   support

긍정(Negative)(0)       0.82      0.90      0.86     79996
부정(Positive)(1)       0.89      0.81      0.85     79998

       accuracy                           0.85    159994
      macro avg       0.86      0.85      0.85    159994
   weighted avg       0.86      0.85      0.85    159994



In [ ]:
# 리뷰 긍/부정 감정 예측 함수
def predict_sentiment_fc(new_review):
    # 입력 텍스트 전처리
    cleaned_review = clean_text(new_review)
    # 텍스트를 벡터화
    new_review_tfidf = tfidf_vectorizer.transform([cleaned_review])
    # 모델로 예측
    prediction = model.predict(new_review_tfidf)[0]
    # 긍/부정일 확률 각각 계산
    prediction_proba = model.predict_proba(new_review_tfidf)[0]
    sentiment = "긍정(Positive)" if prediction == 1 else "부정(Negative)"

    print(f"\n--- 예측 결과 ---")
    print(f"리뷰: '{new_review}'")
    print(f"리뷰 반응: {sentiment}")
    print(f"긍정 확률: {prediction_proba[1]:.4f}, 부정 확률: {prediction_proba[0]:.4f}")
    
    return sentiment, prediction_proba

In [6]:
# -----------------------------------------------------------------------

In [7]:
# 새로운 리뷰 테스트
print("\n--- 새로운 리뷰 예측 테스트 ---")
review1 = "이 영화 정말 최고였어요! 배우들 연기도 좋고 스토리도 완벽합니다."
predict_sentiment_fc(review1)
review2 = "시간 낭비였어요. 지루하고 개연성도 없네요. 추천하지 않습니다."
predict_sentiment_fc(review2)
review3 = "그럭저럭 볼만했습니다. 기대했던 것보다는 아니지만 나쁘지 않았어요."
predict_sentiment_fc(review3)


--- 새로운 리뷰 예측 테스트 ---

--- 예측 결과 ---
리뷰: '이 영화 정말 최고였어요! 배우들 연기도 좋고 스토리도 완벽합니다.'
리뷰 반응: 긍정(Positive)
긍정 확률: 0.9362, 부정 확률: 0.0638

--- 예측 결과 ---
리뷰: '시간 낭비였어요. 지루하고 개연성도 없네요. 추천하지 않습니다.'
리뷰 반응: 부정(Negative)
긍정 확률: 0.0233, 부정 확률: 0.9767

--- 예측 결과 ---
리뷰: '그럭저럭 볼만했습니다. 기대했던 것보다는 아니지만 나쁘지 않았어요.'
리뷰 반응: 긍정(Positive)
긍정 확률: 0.5687, 부정 확률: 0.4313


('긍정(Positive)', array([0.43128783, 0.56871217]))

In [ ]:
# 위 함수를 이용해 크롤링한 영화 리뷰 긍/부정 판결

# 리뷰 데이터 불러오기
reviews_df = pd.read_csv('E:/KOSMO/TeamProject/KOSMO_AI_TP/test_data/MJ/data/reviews.csv')
reviews_df

# 리뷰 데이터에 감정 라벨링
sentiments = []
for review in reviews_df['review']:
    sentiment, Probability = predict_sentiment_fc(review)
    sentiments.append(sentiment)

# 새 컬럼으로 추가
reviews_df['sentiment'] = sentiments
  
# 새 파일로 저장
reviews_df.to_csv('../data/fi_reviews.csv', index=False, encoding='utf-8')


--- 예측 결과 ---
리뷰: '너무많이봐서기억도안나'
리뷰 반응: 부정(Negative)
긍정 확률: 0.4799, 부정 확률: 0.5201

--- 예측 결과 ---
리뷰: '남녀주인공 볼라고 보는 드라마 아님
나는 추자현 연기 볼려고 견우본닼ㅋㅋㅋㅋ 심지어 남녀주인공 건뛰
ㅋㅋㅋㅋㅋ😂연기 찰지게 잘하셔 트리거에서도 몰입도 짱이였는데 👍'
리뷰 반응: 부정(Negative)
긍정 확률: 0.4976, 부정 확률: 0.5024

--- 예측 결과 ---
리뷰: '마왕과나'
리뷰 반응: 부정(Negative)
긍정 확률: 0.4799, 부정 확률: 0.5201

--- 예측 결과 ---
리뷰: '재밌네 ㅋㅋㅋㅋ이현이 보러 가즈아~~~'
리뷰 반응: 긍정(Positive)
긍정 확률: 0.9015, 부정 확률: 0.0985

--- 예측 결과 ---
리뷰: '인형의기사'
리뷰 반응: 부정(Negative)
긍정 확률: 0.4799, 부정 확률: 0.5201

--- 예측 결과 ---
리뷰: '조이현 너무너무 귀엽고 이쁘게 나오고 
너무 재미있어요❤️❤️❤️❤️❤️'
리뷰 반응: 긍정(Positive)
긍정 확률: 0.9471, 부정 확률: 0.0529

--- 예측 결과 ---
리뷰: '귀여운 타란티노_43609'
리뷰 반응: 긍정(Positive)
긍정 확률: 0.8712, 부정 확률: 0.1288

--- 예측 결과 ---
리뷰: '재밌음'
리뷰 반응: 긍정(Positive)
긍정 확률: 0.9863, 부정 확률: 0.0137

--- 예측 결과 ---
리뷰: '즐거운_마동석_209083'
리뷰 반응: 부정(Negative)
긍정 확률: 0.4799, 부정 확률: 0.5201

--- 예측 결과 ---
리뷰: '2화까지 - 이거왜재밌음? 돌풍감독이라 함 보았는데 탄탄하다'
리뷰 반응: 긍정(Positive)
긍정 확률: 0.6864, 부정 확률: 0.3136

--- 예측 결과 ---
리뷰: '상냥한_슈카_128005'
리뷰 반응: 부정(Negative)

In [20]:
review_text = input("리뷰 입력")

# 입력한 리뷰를 모델에 넣어 예측
sentiment, proba = predict_sentiment_fc(review_text)

# 예측 결과 출력
print(f"예측 결과: {sentiment}")

if sentiment == "부정(Negative)":
    print("만족스럽지 않은 콘텐츠였나봐요. 더 좋은 콘텐츠를 추천해드릴게요!")
else :
    print("마음에 꼭 드셨군요!")


--- 예측 결과 ---
리뷰: '별로임'
리뷰 반응: 부정(Negative)
긍정 확률: 0.0315, 부정 확률: 0.9685
예측 결과: 부정(Negative)
만족스럽지 않은 콘텐츠였나봐요. 더 좋은 콘텐츠를 추천해드릴게요!
